In [42]:
import numpy as np
import pandas as pd

from utils.data_transformation import \
    get_train_csr_matrix, get_test_data, compute_metrics

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import \
    precision_at_k, mean_average_precision_at_k

In [1]:
train_ratio = 0.8
user_movies, users_rev_mapping, movies_rev_mapping = get_train_csr_matrix(train_ratio)

als_model = AlternatingLeastSquares(factors=64, regularization=0.05, iterations=20, random_state=42)
als_model.fit(user_movies)

  0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
df_inter = pd.read_csv('processed_data/interactions.csv')
df_items = pd.read_csv('processed_data/items.csv')

users_mapping = {user_id: idx for idx, user_id in users_rev_mapping.items()}
movies_mapping = {movie_id: idx for idx, movie_id in movies_rev_mapping.items()}

id_df_idx_mapping = {id: idx for idx, id in df_items['item_id'].items()}

### Example of making recommendations by AlternatingLeastSquares

In [9]:
test_user_id = df_inter[:int(0.8*len(df_inter))].loc[12321, 'user_id']
df_inter[df_inter['user_id'] == test_user_id].merge(df_items)

,user_id,item_id,last_watch_dt,watched_pct,content_type,title,genres,age_rating,keywords
0,11420,9342,2021-03-13,85.0,film,Дэдпул,"боевики, фантастика, приключения, комедии",18,"антигерой, наемник, супергерой, основанный на ..."
1,11420,12849,2021-03-13,38.0,film,Первый мститель,"боевики, фантастика, приключения",12,"нью-йорк, вторая мировая война, нацисты, супер..."
2,11420,3095,2021-03-13,0.0,film,Тор: Царство тьмы,"боевики, фэнтези, приключения",12,"супергерой, основанный на комическом, скандина..."
3,11420,2280,2021-05-05,25.0,series,На безымянной высоте,"боевики, историческое, военные",16,NaN
4,11420,3734,2021-07-03,0.0,film,Прабабушка легкого поведения,комедии,16,", 2021, россия, прабабушка, легкого, поведения"


In [10]:
test_user_idx = users_mapping[test_user_id]
rec_movies_idx = als_model.recommend(test_user_idx, user_movies[test_user_idx], N=20)[0]

rec_movies_id = np.vectorize(movies_rev_mapping.get)(rec_movies_idx)
rec_movies_id = np.vectorize(id_df_idx_mapping.get)(rec_movies_id)
df_items.iloc[rec_movies_id].head()

,item_id,content_type,title,genres,age_rating,keywords
1547,10942,film,Мстители,"боевики, фантастика, фэнтези, приключения",12,"нью-йорк, щит, супергерой, основанный на комик..."
11782,7210,film,Тор: Рагнарёк,"приключения, фантастика, боевики, фэнтези, ком...",16,"продолжение, супергерой, основанный на комиксе..."
2638,12841,film,Стражи Галактики,"боевики, фантастика, приключения, комедии",12,"космический корабль, основанный на комиксе, ко..."
13161,3587,film,Железный человек,"боевики, фантастика, приключения",12,"Ближний Восток, торговец оружием, Малибу, супе..."
5443,682,film,Мстители: Эра Альтрона,"боевики, фантастика, приключения",12,"продолжение, супергерой, основанный на комикса..."


### Metrics

In [50]:
test_data = get_test_data(train_ratio=0.8, watch_threshold=5)

test_interactions = {}
for user_id, item_id in test_data[['user_id', 'item_id']].values:
    user_idx = users_mapping[user_id]
    if user_idx not in test_interactions:
        test_interactions[user_idx] = set()
    test_interactions[user_idx].add(movies_mapping[item_id])

users_idx = np.array(list(test_interactions.keys()))
recommendations = als_model.recommend(users_idx, user_movies[users_idx], N=20)[0]

precision, recall = compute_metrics(users_idx, test_interactions, recommendations)
precision, recall

(0.017611266748635, 0.08008239974441751)

In [40]:
from scipy.sparse import csr_matrix

users = test_data['user_id'].map(users_mapping)
movies = test_data['item_id'].map(movies_mapping)

test_user_movies = csr_matrix(
    (np.ones(len(test_data)), (users, movies)),
    shape=(len(users_mapping), len(movies_mapping)))

In [46]:
precision = precision_at_k(als_model, user_movies, test_user_movies, K=20)
map = mean_average_precision_at_k(als_model, user_movies, test_user_movies, K=20)

precision, map

  0%|          | 0/78887 [00:00<?, ?it/s]

  0%|          | 0/78887 [00:00<?, ?it/s]

(0.08199096464644974, 0.023250566705251323)